<a href="https://colab.research.google.com/github/mmutyampeta/EduML/blob/main/RidgeRegression_EduML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

dataset = pd.read_csv("2015_16_DistrictwiseNEW.csv")
print(dataset.shape)

#prepare the data into the right shapes, and inserting mean values in place of NaN values
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(strategy='mean')
dataset = imp_mean.fit_transform(dataset)
dataset = pd.DataFrame(dataset)
X = dataset.loc[:, dataset.columns != 'OVERALL_LI']
y = dataset.iloc[:,11]
X = np.array(X)
y = np.array(y)
y = y.reshape(-1,1)

columns = list(dataset.columns)
features1 = columns[1:11]
features2 = columns[12:]
features = features1 + features2

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

(680, 813)


In [21]:
print(features)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 2

In [22]:
from sklearn.linear_model import RidgeCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
cv_alphas = [0.001,0.003,0.01,0.03,0.1,0.3,1,3,10] #for cross validation in Ridge
model = RidgeCV(alphas = cv_alphas)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
score = r2_score(y_test,y_pred)
print(score)

MSE = mean_squared_error(y_test,y_pred)
print(MSE)

0.39370503937224843
76.45937994291333


In [23]:
from sklearn.inspection import permutation_importance
r = permutation_importance(model, X_test, y_test,
                          n_repeats=30,
                          random_state=0)

best_features = []
for i in r.importances_mean.argsort()[::-1]:
  if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
    if r.importances_mean[i] > 0.55:
      print(f"{features[i]:<8}"
        f"{r.importances_mean[i]:.3f}"
        f" +/- {r.importances_std[i]:.3f}")
      best_features.append(features[i])

print(best_features)
print(len(best_features))

364     838.368 +/- 86.635
357     358.557 +/- 33.212
752     206.399 +/- 17.918
358     204.700 +/- 17.618
116     151.574 +/- 13.666
746     119.180 +/- 10.799
105     118.393 +/- 11.431
497     94.999 +/- 10.563
125     80.690 +/- 7.388
362     66.341 +/- 2.510
108     59.355 +/- 4.955
499     58.542 +/- 6.249
109     46.835 +/- 3.296
104     34.502 +/- 1.612
360     33.608 +/- 2.823
750     32.943 +/- 2.274
103     30.457 +/- 2.074
759     29.555 +/- 3.979
361     22.473 +/- 1.591
99      21.003 +/- 1.930
586     16.732 +/- 1.641
359     16.291 +/- 1.401
749     16.038 +/- 1.369
439     15.642 +/- 1.479
117     15.258 +/- 1.373
363     14.972 +/- 1.515
446     13.218 +/- 1.471
227     13.164 +/- 1.168
112     12.823 +/- 0.929
123     11.755 +/- 1.115
437     11.443 +/- 1.094
751     11.203 +/- 0.407
98      9.801 +/- 0.880
134     9.577 +/- 0.727
340     9.311 +/- 0.971
341     9.133 +/- 0.727
498     8.399 +/- 0.869
102     7.988 +/- 0.738
440     7.849 +/- 0.890
118     7.178 +/-

In [29]:
BestVar = dataset.loc[:,best_features]
print(BestVar.shape)
X_trainN, X_testN, y_trainN, y_testN = train_test_split(BestVar, y, test_size=0.2, random_state=0) 

(680, 139)


In [33]:
modelNew = RidgeCV(alphas = cv_alphas)
modelNew.fit(X_trainN,y_trainN)
y_predN = modelNew.predict(X_testN)
scoreN = r2_score(y_testN,y_predN)
print(scoreN)

MSE = mean_squared_error(y_test,y_pred)
print(MSE)

-437.6419091250182
76.45937994291333
